In [1]:
import random


def play(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (
                p1_play == "R" and p2_play == "S") or (p1_play == "S"
                                                       and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return (win_rate)


def quincy(prev_play, counter=[0]):

    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]


def mrugesh(prev_opponent_play, opponent_history=[]):
    opponent_history.append(prev_opponent_play)
    last_ten = opponent_history[-10:]
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == '':
        most_frequent = "S"

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[most_frequent]


def kris(prev_opponent_play):
    if prev_opponent_play == '':
        prev_opponent_play = "R"
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prev_opponent_play]


def abbey(prev_opponent_play,
          opponent_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not prev_opponent_play:
        prev_opponent_play = 'R'
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order[0]
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]


def human(prev_opponent_play):
    play = ""
    while play not in ['R', 'P', 'S']:
        play = input("[R]ock, [P]aper, [S]cissors? ")
        print(play)
    return play


def random_player(prev_opponent_play):
    return random.choice(['R', 'P', 'S'])


In [2]:
import tensorflow as tf
import tensorflow.keras as keras

def encodeRPS(char):
    if char == 'R':
        return 0
    elif char == 'P':
        return 1
    elif char == 'S':
        return 2
    
def decodeRPS(num):
    if num == 0:
        return 'R'
    elif num == 1:
        return 'P'
    elif num == 2:
        return 'S'  

def Get_Prediction(model, opponent_history):
    #format history for input
    input_eval = [0 if c == '' else encodeRPS(c) for c in opponent_history]
    input_eval = tf.expand_dims(input_eval, 0)
    
    #high temperature  => suprising output
    #low temperature => predictable output
    temperature = 0.1; 
    
    model.reset_states()
    
    prediction = model(input_eval)
    prediction = tf.squeeze(prediction, 0)
    
    prediction = prediction / temperature
    predicted_id = tf.random.categorical(prediction, num_samples = 1)[-1,0].numpy()
    
    return decodeRPS(predicted_id)

In [3]:
# Test Loading
model_test = keras.models.load_model('Models/LSTM_Random_and_Ideal_InputBased_All_Abbey')

In [4]:
model_test.summary()
print(model_test.get_weights())
print(model_test.optimizer)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 8)              24        
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           4231168   
_________________________________________________________________
dense_1 (Dense)              (1, None, 4)              4100      
Total params: 4,235,292
Trainable params: 4,235,292
Non-trainable params: 0
_________________________________________________________________
[array([[ 0.16155487,  0.34387758, -0.67708224, -0.58314   , -0.6192922 ,
        -0.49831024,  0.6406542 , -0.39273107],
       [ 0.5828614 ,  0.34342846,  0.03933175,  0.2791612 ,  0.18513541,
         0.5085075 , -0.17405964,  0.93006617],
       [-0.6878341 , -0.615081  ,  0.39825636,  0.27618375,  0.36525542,
         0.10149401, -0.31026828, -0.48526138]], dtype=flo

In [5]:
print(Get_Prediction(model_test, ['R','R','P']))
plays = 1000
def player_test(prev_play, opponent_history=[], player_history=['']):
    if len(opponent_history) >= plays: 
        opponent_history.clear()
        player_history.clear()
        player_history.append('')
        
    opponent_history.append(prev_play)
    
    guess = Get_Prediction(model_test, player_history)

    #print(len(opponent_history))
    player_history.append(guess)
    return guess

S


In [6]:
play(player_test, quincy, plays)
play(player_test, abbey, plays)
play(player_test, kris, plays)
play(player_test, mrugesh, plays)

Final results: {'p1': 333, 'p2': 332, 'tie': 335}
Player 1 win rate: 50.07518796992482%
Final results: {'p1': 777, 'p2': 112, 'tie': 111}
Player 1 win rate: 87.4015748031496%
Final results: {'p1': 333, 'p2': 334, 'tie': 333}
Player 1 win rate: 49.92503748125937%
Final results: {'p1': 335, 'p2': 333, 'tie': 332}
Player 1 win rate: 50.1497005988024%


50.1497005988024

In [7]:
play(player_test, abbey, plays)

Final results: {'p1': 777, 'p2': 112, 'tie': 111}
Player 1 win rate: 87.4015748031496%


87.4015748031496